In [15]:
import pandas as pd
# load data and merge features and label
df_meta = pd.read_csv('../../preprocessing/datasets/tracks_with_genre_small.csv')
df_features = pd.read_csv('../../feature_extraction/features/all_features.csv')
merged_df = df_meta.merge(df_features, on='track_id', how='outer')

In [20]:
# drop unnecessary column and rows including null values)
LABEL = 'depth_1_genre_name'
used_columns = [LABEL] + [col for col in df_features.columns if col != 'Unnamed: 0']
merged_df = merged_df[used_columns].dropna()

merged_df.head()



,depth_1_genre_name,track_id,tempo,rms_harmonic,rms_percussive,spectral_centroid,spectral_rolloff,zero_crossings,chroma_stft_1,chroma_stft_2,...,MFCC_11,MFCC_12,MFCC_13,MFCC_14,MFCC_15,MFCC_16,MFCC_17,MFCC_18,MFCC_19,MFCC_20
0,Hip-Hop,2,161.499023,0.060801,0.102429,2975.279504,6088.350559,107473.0,0.747086,0.465628,...,-13.843452,6.846233,-6.376837,11.474691,-6.496868,4.351640,-11.482881,0.858946,-5.743586,2.784478
1,Hip-Hop,5,99.384014,0.079553,0.085417,2361.216889,5326.953068,65573.0,0.408813,0.578352,...,-4.723806,11.691971,0.694524,9.956776,-2.353443,3.412343,-6.521324,1.716783,-3.797788,3.443657
2,Pop,10,112.347147,0.135624,0.081036,2319.325849,4256.393704,98089.0,0.283689,0.649424,...,-12.372822,5.297867,-9.403314,5.570858,-7.896959,1.831656,-5.827612,5.577751,-3.469359,6.202434
3,Folk,140,107.666016,0.060838,0.015485,1680.622215,3679.342224,28713.0,0.243197,0.271314,...,-6.952166,0.507389,-8.884753,4.870484,-10.186780,3.072841,-6.979950,0.914813,-7.929453,0.331834
4,Folk,141,117.453835,0.089626,0.023680,1611.748689,3238.053335,40485.0,0.164148,0.153275,...,-4.950849,-1.830327,-6.346319,4.617892,-3.517620,6.454854,-1.449563,4.338287,-9.859125,0.000951


In [21]:
# Normalize feature values.
from sklearn.preprocessing import MinMaxScaler

feature_list = [col for col in df_features.columns if col not in ['Unnamed: 0', 'track_id']]
scaler = MinMaxScaler()

merged_df[feature_list] = scaler.fit_transform(merged_df[feature_list])
merged_df.head()

,depth_1_genre_name,track_id,tempo,rms_harmonic,rms_percussive,spectral_centroid,spectral_rolloff,zero_crossings,chroma_stft_1,chroma_stft_2,...,MFCC_11,MFCC_12,MFCC_13,MFCC_14,MFCC_15,MFCC_16,MFCC_17,MFCC_18,MFCC_19,MFCC_20
0,Hip-Hop,2,0.750000,0.088395,0.281318,0.451822,0.660061,0.263980,0.747086,0.468934,...,0.403114,0.691606,0.462157,0.719704,0.520207,0.586388,0.423355,0.493065,0.429066,0.420074
1,Hip-Hop,5,0.461538,0.115659,0.234596,0.358571,0.577515,0.161062,0.408813,0.582458,...,0.514440,0.764306,0.579668,0.693797,0.587037,0.571558,0.502464,0.511341,0.463714,0.430719
2,Pop,10,0.521739,0.197177,0.222562,0.352210,0.461451,0.240930,0.283689,0.654035,...,0.421067,0.668376,0.411864,0.618943,0.497624,0.546601,0.513525,0.593599,0.469562,0.475272
3,Folk,140,0.500000,0.088449,0.042528,0.255217,0.398891,0.070524,0.243197,0.273240,...,0.487238,0.596505,0.420481,0.606989,0.460691,0.566197,0.495151,0.494255,0.390144,0.380465
4,Folk,141,0.545455,0.130302,0.065037,0.244758,0.351049,0.099439,0.164148,0.154363,...,0.511669,0.561432,0.462664,0.602678,0.568260,0.619594,0.583330,0.567192,0.355783,0.375121


In [25]:
merged_df = merged_df.set_index('track_id')

In [32]:
# get similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(merged_df[feature_list])
labels = merged_df['depth_1_genre_name']
result = pd.DataFrame(similarity, index=labels.index, columns=labels.index)
result

track_id,2,5,10,140,141,148,182,190,193,194,...,154308,154308,154308,154309,154309,154309,154413,154414,155066,155066
track_id,,,,,,,,,,,,,,,,,,,,,
2,1.000000,0.979764,0.959661,0.946836,0.939294,0.942771,0.952282,0.956872,0.954705,0.959740,...,0.953305,0.953305,0.953305,0.955173,0.955173,0.955173,0.960249,0.958936,0.916577,0.916577
5,0.979764,1.000000,0.978421,0.964308,0.959948,0.963156,0.966781,0.967418,0.973327,0.977470,...,0.974229,0.974229,0.974229,0.966140,0.966140,0.966140,0.970826,0.973418,0.952652,0.952652
10,0.959661,0.978421,1.000000,0.949714,0.953569,0.963517,0.963805,0.955149,0.965110,0.960218,...,0.962307,0.962307,0.962307,0.952891,0.952891,0.952891,0.949631,0.952411,0.935865,0.935865
140,0.946836,0.964308,0.949714,1.000000,0.982654,0.952442,0.955626,0.974124,0.973134,0.978479,...,0.955785,0.955785,0.955785,0.963631,0.963631,0.963631,0.978691,0.984075,0.941594,0.941594
141,0.939294,0.959948,0.953569,0.982654,1.000000,0.962898,0.960428,0.969544,0.954621,0.971048,...,0.954452,0.954452,0.954452,0.947294,0.947294,0.947294,0.963520,0.967638,0.947097,0.947097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154309,0.955173,0.966140,0.952891,0.963631,0.947294,0.951745,0.949419,0.970667,0.955803,0.961687,...,0.947658,0.947658,0.947658,1.000000,1.000000,1.000000,0.965587,0.974487,0.930636,0.930636
154413,0.960249,0.970826,0.949631,0.978691,0.963520,0.957599,0.959141,0.964379,0.966891,0.974569,...,0.962038,0.962038,0.962038,0.965587,0.965587,0.965587,1.000000,0.973686,0.960068,0.960068
154414,0.958936,0.973418,0.952411,0.984075,0.967638,0.958205,0.965621,0.985292,0.977470,0.983671,...,0.958542,0.958542,0.958542,0.974487,0.974487,0.974487,0.973686,1.000000,0.937610,0.937610
